In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import json
import pickle

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset, random_split

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report

In [5]:
USE_CUDA = torch.cuda.is_available()
device = "cuda" if USE_CUDA else "cpu"

## Данные

In [6]:
data = pd.read_csv('data/dataset.csv') # болезнь - симптомы
df_description = pd.read_csv('data/symptom_Description.csv') # болезнь - описание
df_precaution = pd.read_csv('data/symptom_precaution.csv') # болезнь - рекомендации
df_severity = pd.read_csv('data/Symptom-severity.csv') # симпом - вес

In [7]:
di_description = {val[0]: val[1] for val in df_description.values}
di_precaution = {val[0]: list(val[1:5]) for val in df_precaution.values}

In [8]:
data.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Обработка

In [9]:
df_severity.Symptom.value_counts()

fluid_overload          2
itching                 1
spinning_movements      1
muscle_pain             1
irritability            1
                       ..
constipation            1
back_pain               1
pain_behind_the_eyes    1
loss_of_appetite        1
prognosis               1
Name: Symptom, Length: 132, dtype: int64

In [10]:
df_severity[df_severity.Symptom == 'fluid_overload']

,Symptom,weight
45,fluid_overload,6
117,fluid_overload,4


In [11]:
df_severity.drop([117], inplace=True)

In [12]:
df_severity.reset_index(inplace=True, drop=True)

In [13]:
df_severity.iloc[115:120]

,Symptom,weight
115,distention_of_abdomen,4
116,history_of_alcohol_consumption,5
117,blood_in_sputum,5
118,prominent_veins_on_calf,6
119,palpitations,4


In [14]:
df_severity.loc[df_severity.Symptom == 'fluid_overload', 'weight'] = 5

In [15]:
df_severity[df_severity.Symptom == 'fluid_overload']

,Symptom,weight
45,fluid_overload,5


In [16]:
data_copy = data.copy()

In [17]:
data.iloc[:,1:] = [[symp.strip().replace(' ','') if type(symp) != float else symp for symp in row] for row in data_copy.iloc[:,1:].values]

In [18]:
data_copy.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
data.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic_patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic_patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic_patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic_patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Замена симптомов весами

In [20]:
df_severity.head()

,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5


In [21]:
di_severity_weights = {row[0]:row[1] for row in df_severity.values}

In [22]:
del di_severity_weights['prognosis']

In [23]:
list_di_severity_weights = list(di_severity_weights.keys())
list_di_severity_weights.sort()

In [24]:
all_symps_in_data = []
for symps in data.iloc[:,1:].apply(lambda col: col.unique()):
    for symp in symps:
        if type(symp) != float:
            all_symps_in_data.append(symp)

In [25]:
list_data_symps = list(set(all_symps_in_data))
list_data_symps.sort()

In [26]:
df_severity[df_severity.eq('prognosis').any(1)]

,Symptom,weight
131,prognosis,5


In [27]:
for i,j in zip(list_di_severity_weights, list_data_symps):
    if i != j:
        i, j

In [28]:
di_severity_weights = {row[0]:row[1] for row in df_severity.values}

In [29]:
data.replace(di_severity_weights, inplace=True)

In [30]:
data.fillna(0, inplace=True)

In [31]:
data.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,1,3,4,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Fungal infection,3,4,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Fungal infection,1,4,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Fungal infection,1,3,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Fungal infection,1,3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
data.iloc[:,1:] = data.iloc[:,1:].astype('int')

## Модель

In [33]:
X = data.drop(columns=['Disease'])
y = data['Disease']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2, 
                                                    stratify=y.values, shuffle=True, random_state=123)

In [35]:
random_forest = RandomForestClassifier()
random_forest.fit(X_train, y_train)

RandomForestClassifier()

In [36]:
y_pred = random_forest.predict(X_test)
print(f'F1-score = {round(f1_score(y_test, y_pred, average="macro")*100,2)}%\nAccuracy = {round(accuracy_score(y_test, y_pred)*100,2)}%')

F1-score = 99.6%
Accuracy = 99.59%


## RUS

### Симптомы

In [37]:
ru_symptoms = ['боль в животе','ненормальная_менструация','кислотность','острая_печеночная_недостаточность','измененный_сенсориум',
               'беспокойство','боль в спине','боль в брюшной полости','черные точки','дискомфорт_мочевого пузыря','волдырь',
               'кровь_в_мокроте','кровавый_стул','размытое_и_искаженное_зрение','одышка','ломкие_ногти','синяки','жжение при мочеиспускании',
               'боль в груди','озноб','холодные_руки_и_ноги','кома','заложенность носа','запор','Постоянное ощущение мочи',
               'Непрерывное чихание','кашель','судороги','темная_моча','обезвоживание','депрессия','диарея','Дисхромные пятна',
               'растяжение_живота','головокружение','сохнущие_и_покалывающие_губы','увеличенная_щитовидная железа','чрезмерный_голод',
               'внебрачные связи','родословная','Быстрый сердечный ритм','усталость','перегрузка_жидкостью','неприятный_запах_мочи',
               'головная боль','высокая температура','боль в тазобедренном суставе','давнее_потребление_алкоголя','повышенный_аппетит',
               'несварение','воспалительные_ногти','внутренний_зуд','нерегулярный_уровень_сахара','раздражительность','раздражение_в_анусе',
               'зуд','суставная_боль','боль в колене','нехватка концентрации','летаргия','потеря аппетита','потеря_баланса','потеря_запаха',
               'недомогание','слабая лихорадка','перепады настроения','скованность движений','слизистая_мокрота','боли в мышцах',
               'атрофия мышц','мышечная слабость','тошнота','боль в шее','Узловые высыпания на коже','ожирение','боль_за_глазами',
               'боль во время дефекации','боль_в_анальной_области','болезненная_ходьба','учащенное сердцебиение','проход_газов',
               'пятна в горле','мокрота','полиурия','выступающие вены на икрах','одутловатое_лицо_и_глаза','гнойные прыщи',
               'получение_переливания_крови','получение_нестерильных_инъекций','красная язва вокруг носа','красные_пятна_поверх_тела',
               'покраснение_глаз','беспокойство','насморк','ржавая_мокрота','суетливость','дрожь','Аргироз','давление пазухи','шелушение кожи',
               'кожная_сыпь','невнятная речь','небольшие вмятины на ногтях','вращательные движения','обнаружение_мочеиспускания','скованность мышц шеи',
               'желудочное кровотечение','боль в желудке','запавшие глаза','потливость','опухшие_лимфатические_узлы','отек_суставов',
               'опухание_желудка','опухшие_кровяные_сосуды','опухшие_конечности','опухшие_ноги','раздражение горла','ядовитый взгляд_(тиф)',
               'язвы_на_языке','неустойчивость','визуальные_нарушения','рвота','слезотечение','слабость_в_конечностях','слабость_одной_стороны_тела',
               'увеличение веса','потеря веса','желтая корочка','желтая_моча','пожелтение_глаз','желтоватая_кожа']

In [38]:
ru_symptoms = [s.capitalize().replace('_',' ').strip() for s in ru_symptoms]

In [39]:
symp_eng_to_ru = {x[0]:x[1] for x in zip(list_data_symps,ru_symptoms)}
symp_ru_to_eng = {x[0]:x[1] for x in zip(ru_symptoms,list_data_symps)}

### Болезни

In [40]:
ru_diseases = ['Лекарственная реакция', 'Малярия', 'Аллергия', 'Гипотиреоз',
               'Псориаз', 'ГЭРБ', 'Хронический холестаз', 'Гепатит А',
               'Остеоартроз', '(головокружение) Пароймсальное позиционное головокружение',
               'Гипогликемия', 'Угри', 'Диабет', 'Импетиго', 'Гипертония',
               'Язвенная болезнь', 'Диморфный геморрой',
               'Простуда', 'Ветряная оспа', 'Шейный спондилез',
               'Гипертиреоз', 'Инфекция мочевыводящих путей', 'Варикозное расширение вен',
               'СПИД', 'Паралич (кровоизлияние в мозг)', 'Брюшной тиф', 'Гепатит В',
               'Грибковая инфекция', 'Гепатит С', 'Мигрень', 'Бронхиальная астма',
               'Алкогольный гепатит', 'Желтуха', 'Гепатит Е', 'Лихорадка денге',
               'Гепатит D', 'Сердечный приступ', 'Пневмония', 'Артрит',
               'Гастроэнтерит', 'Туберкулез']

In [41]:
disease_eng_to_ru = {x[0]:x[1] for x in zip(list(df_description.Disease),ru_diseases)}
disease_ru_to_eng = {x[0]:x[1] for x in zip(ru_diseases,list(df_description.Disease))}

### Описания

In [42]:
ru_descr = ['Нежелательная лекарственная реакция (ПНР) — это травма, вызванная приемом лекарств. Побочные реакции могут возникать после однократного или длительного приема препарата или в результате комбинации двух или более препаратов.',
  'Инфекционное заболевание, вызываемое простейшими паразитами из семейства Plasmodium, которое может передаваться при укусе комара Anopheles, зараженной игле или переливании крови. Falciparum малярия является наиболее смертоносным типом.',
  'Аллергия — это реакция иммунной системы на чужеродное вещество, которое обычно не вредно для вашего организма. Это могут быть определенные продукты, пыльца или шерсть домашних животных. Задача вашей иммунной системы — поддерживать ваше здоровье, борясь с вредными патогенами',
  'Гипотиреоз, также называемый недостаточной активностью щитовидной железы или низким уровнем щитовидной железы, представляет собой нарушение эндокринной системы, при котором щитовидная железа не вырабатывает достаточное количество гормона щитовидной железы.',
  'Псориаз — это распространенное кожное заболевание, при котором образуются толстые, красные, неровные пятна, покрытые серебристыми чешуйками. Они могут появиться где угодно, но чаще всего появляются на коже головы, локтях, коленях и нижней части спины. Псориаз не передается от человека к человеку. Иногда это случается у членов одной семьи.',
  'Гастроэзофагеальная рефлюксная болезнь, или ГЭРБ, — это расстройство пищеварения, которое поражает нижний пищеводный сфинктер (НПС), мышечное кольцо между пищеводом и желудком. Многие люди, в том числе беременные женщины, страдают от изжоги или кислотного расстройства желудка, вызванного ГЭРБ.',
  'Хронические холестатические заболевания, возникающие в младенчестве, детстве или во взрослом возрасте, характеризуются нарушением транспорта желчных кислот из печени в кишечник, что в большинстве случаев обусловлено первичным повреждением билиарного эпителия',
  'Гепатит А — это высококонтагиозная инфекция печени, вызываемая вирусом гепатита А. Этот вирус является одним из нескольких типов вирусов гепатита, которые вызывают воспаление и влияют на способность вашей печени функционировать',
  'Остеоартрит — наиболее распространенная форма артрита, поражающая миллионы людей во всем мире. Это происходит, когда защитный хрящ, покрывающий концы костей, со временем изнашивается.',
  'Доброкачественное пароксизмальное позиционное головокружение (ДППГ) является одной из наиболее частых причин головокружения — внезапное ощущение, что вы вращаетесь или что внутри вашей головы вращается. Доброкачественное пароксизмальное позиционное головокружение вызывает короткие эпизоды головокружения от легкого до сильного',
  'Гипогликемия — это состояние, при котором уровень сахара (глюкозы) в крови ниже нормы. Глюкоза является основным источником энергии вашего организма. Гипогликемия часто связана с лечением диабета. уровень сахара в крови у людей, у которых нет диабета',
  'Вульгарные угри — это образование комедонов, папул, пустул, узелков и/или кист в результате закупорки и воспаления сально-волосяных единиц (волосяных фолликулов и сопровождающих их сальных желез). Акне развивается на лице и верхней части туловища. Чаще всего поражает подростков.',
  'Диабет — это заболевание, которое возникает, когда уровень глюкозы в крови, также называемый сахаром в крови, слишком высок. Глюкоза в крови является вашим основным источником энергии и поступает из пищи, которую вы едите. Инсулин, гормон, вырабатываемый поджелудочной железой, помогает глюкозе из пищи попадать в ваши клетки и использоваться для получения энергии.',
  'Импетиго (im-puh-TIE-go) — распространенная и очень заразная кожная инфекция, поражающая в основном младенцев и детей. Импетиго обычно проявляется в виде красных язв на лице, особенно вокруг носа и рта ребенка, а также на руках и ногах. Язвы лопаются и покрываются корками медового цвета',
  'Гипертония (HTN или HT), также известная как высокое кровяное давление (HBP), представляет собой длительное заболевание, при котором кровяное давление в артериях постоянно повышено. Высокое кровяное давление обычно не вызывает симптомов.',
  'Язвенная болезнь (ЯБ) — это повреждение внутренней оболочки желудка, начального отдела тонкой кишки или иногда нижнего отдела пищевода. Язва в желудке называется язвой желудка, а язва в первом отделе кишечника - язвой двенадцатиперстной кишки.',
  'Геморрой, также называемый геморроем, представляет собой сосудистые образования в анальном канале. В их... Другие названия, Геморрой, сваи, геморроидальная болезнь.',
  'Простуда — это вирусная инфекция носа и горла (верхних дыхательных путей). Обычно она безвредна, хотя может и не ощущаться так. Многие типы вирусов могут вызывать простуду',
  'Ветряная оспа — высококонтагиозное заболевание, вызываемое вирусом ветряной оспы (VZV). Это может вызвать зудящую сыпь, похожую на волдыри. Сыпь сначала появляется на груди, спине и лице, а затем распространяется по всему телу, вызывая от 250 до 500 зудящих волдырей.',
  'Шейный спондилез — это общий термин для обозначения возрастного износа позвоночных дисков шеи. По мере обезвоживания и сморщивания дисков развиваются признаки остеоартрита, в том числе костные выступы по краям костей (костные шпоры).',
  'Гипертиреоз (сверхактивная щитовидная железа) возникает, когда щитовидная железа вырабатывает слишком много гормона тироксин. Гипертиреоз может ускорить обмен веществ в организме, вызывая непреднамеренную потерю веса и учащенное или нерегулярное сердцебиение.',
  'Инфекция мочевыводящих путей: инфекция почек, мочеточников, мочевого пузыря или уретры. Сокращенно ИМП. Не у всех с ИМП есть симптомы, но общие симптомы включают частые позывы к мочеиспусканию и боль или жжение при мочеиспускании.',
  'Вена, которая увеличилась и перекрутилась, часто выглядит как выпуклый голубой кровеносный сосуд, который хорошо виден через кожу. Варикозное расширение вен чаще всего встречается у пожилых людей, особенно у женщин, особенно на ногах.',
  'Синдром приобретенного иммунодефицита (СПИД) — это хроническое, потенциально опасное для жизни состояние, вызванное вирусом иммунодефицита человека (ВИЧ). Повреждая вашу иммунную систему, ВИЧ препятствует способности вашего организма бороться с инфекциями и болезнями',
  'Внутримозговое кровоизлияние (ВМК) — это когда кровь внезапно прорывается в мозговую ткань, вызывая повреждение мозга. Симптомы обычно появляются внезапно во время ВЧК. Они включают головную боль, слабость, спутанность сознания и паралич, особенно на одной стороне тела.',
  'Острое заболевание, характеризующееся лихорадкой, вызванной инфекцией бактерией Salmonella typhi. Брюшной тиф начинается незаметно, с лихорадки, головной боли, запоров, недомогания, озноба и мышечных болей. Диарея бывает редко, а рвота обычно нетяжелая.',
  'Гепатит В — это инфекция печени. Он может вызвать рубцевание органа, печеночную недостаточность и рак. Если его не лечить, он может привести к летальному исходу. Он распространяется при контакте людей с кровью, открытыми язвами или биологические жидкости человека, у которого есть вирус гепатита В.',
  'У людей грибковые инфекции возникают, когда вторгшийся грибок захватывает часть тела, и иммунная система не может с ним справиться. Грибы могут жить в воздухе, почве, воде и растениях. Есть также некоторые грибы, которые естественным образом живут в организме человека. Как и многие микробы, существуют полезные и вредные грибы.',
  'Воспаление печени, вызванное вирусом гепатита С (HCV), который обычно распространяется через переливание крови (редко), гемодиализ и уколы иглами. Вред, который гепатит С наносит печени, может привести к циррозу и его осложнениям, а также к раку.',
  'Мигрень может вызывать сильную пульсирующую боль или ощущение пульсации, обычно с одной стороны головы. Она часто сопровождается тошнотой, рвотой и повышенной чувствительностью к свету и звуку. Приступы мигрени могут длиться от нескольких часов до нескольких дней, а боль может быть настолько серьезным, чтобы это мешало вашей повседневной деятельности',
  'Бронхиальная астма — это заболевание, при котором дыхательные пути легких отекают и сужаются. Из-за этого отека в дыхательных путях образуется избыток слизи, что затрудняет дыхание, что приводит к кашлю, одышке и свистящему дыханию. Заболевание хроническое и мешает повседневной работе.',
  'Алкогольный гепатит — это болезненное воспалительное состояние печени, вызванное чрезмерным употреблением алкоголя в течение длительного периода времени. Он также усугубляется запоями и постоянным употреблением алкоголя. Если у вас развивается это состояние, вы должны прекратить употребление алкоголя',
  'Желтое окрашивание кожи и склер (белков глаз) из-за аномально высокого уровня в крови желчного пигмента билирубина. Пожелтение распространяется на другие ткани и жидкости организма. Когда-то желтуху называли "morbus regius" ("царская болезнь") из-за убеждения, что только прикосновение короля может вылечить ее.',
  'Редкая форма воспаления печени, вызванная заражением вирусом гепатита Е (HEV). Он передается через пищу или питье, которыми пользовался инфицированный человек, или через зараженные источники воды в местах, где фекалии могут попасть в воду. Гепатит Е не вызывает хронического заболевания печени.',
  'острое инфекционное заболевание, вызываемое флавивирусом (видом вируса денге рода Flavivirus), передающееся комарами рода Aedes, характеризующееся головной болью, сильной болью в суставах и сыпью. — называемая также лихорадкой переломов, лихорадкой денге.',
  'Гепатит D, также известный как вирус гепатита дельта, представляет собой инфекцию, вызывающую воспаление печени. Этот отек может ухудшить функцию печени и вызвать долгосрочные проблемы с печенью, включая рубцевание печени и рак. Состояние вызвано вирусом гепатита D (HDV).',
  'Гибель сердечной мышцы из-за потери кровоснабжения. Потеря кровоснабжения обычно вызвана полной закупоркой коронарной артерии, одной из артерий, снабжающих кровью сердечную мышцу.',
  'Пневмония — это инфекция в одном или обоих легких. Его вызывают бактерии, вирусы и грибки. Инфекция вызывает воспаление в воздушных мешочках в легких, которые называются альвеолами. Альвеолы заполняются жидкостью или гноем, что затрудняет дыхание.',
  'Артрит — это отек и болезненность одного или нескольких суставов. Основными симптомами артрита являются боль в суставах и скованность, которые обычно усиливаются с возрастом. Наиболее распространенными типами артрита являются остеоартрит и ревматоидный артрит.',
  'Гастроэнтерит – это воспаление пищеварительного тракта, особенно желудка, толстого и тонкого кишечника. Вирусный и бактериальный гастроэнтерит — это кишечные инфекции, сопровождающиеся симптомами диареи, спазмов в животе, тошноты и рвоты.',
  'Туберкулез (ТБ) — это инфекционное заболевание, обычно вызываемое бактериями Mycobacterium tuberculosis (MTB). Туберкулез обычно поражает легкие, но может поражать и другие части тела. Большинство инфекций не проявляют симптомов, и в этом случае они известны как латентный туберкулез']

In [43]:
descr_eng_to_ru = {x[0]:x[1] for x in zip(list(df_description.Description),ru_descr)}
descr_ru_to_eng = {x[0]:x[1] for x in zip(ru_descr,list(df_description.Description))}

### Рекомендации

In [44]:
all_precs = [x for row in df_precaution.iloc[:,1:].values for x in row if type(x) != float]
all_precs = list(set(all_precs))
all_precs.sort()

In [45]:
ru_precs = ['Обратитесь в ближайшую больницу',
  'ацетаминофен',
  'лекарство против зуда',
  'антибиотическая терапия',
  'нанести каламин',
  'избегайте резких движений головой',
  'избегайте холодной пищи',
  'избегайте жирной острой пищи',
  'избегайте лежания после еды',
  'избегайте невегетарианской пищи',
  'избегайте жирной пищи',
  'избегать открытых порезов',
  'избегайте общественных мест',
  'избегайте внезапных изменений в теле',
  'избегайте слишком большого количества продуктов',
  'купаться дважды',
  'вызовите скорую помощь',
  'проверить пульс',
  'жевать или глотать аспирин',
  'холодные ванны',
  'проконсультируйтесь со специализированным врачом',
  'проконсультируйтесь в ближайшей больнице',
  'пить сок алоэ вера',
  'употреблять расторопшу',
  'употреблять листья нима',
  'употребляйте пробиотические продукты',
  'употребить гамамелис',
  'область покрытия повязкой',
  'прикрыть рот',
  'не стой долго на месте',
  'пить клюквенный сок',
  'пить сок листьев папайи',
  'пить много воды',
  'пить сладкие напитки',
  'пить напитки, богатые витамином С',
  'легко возвращаться к еде',
  'ешьте фрукты и продукты с высоким содержанием клетчатки',
  'здоровое питание',
  'ешьте высококалорийные овощи',
  'исключить молоко',
  'упражнения',
  'следовать рекомендациям',
  'избавиться от раздражителя',
  'выспаться',
  'иметь сбалансированную диету',
  'увеличить потребление витамина С',
  'сохранять спокойствие',
  'держать лихорадку под контролем',
  'следить за обезвоживанием',
  'Держите зараженную область сухой',
  'отпугивать комаров',
  'не пускать комаров',
  'лечь',
  'лечь ровно и высоко поднять ногу',
  'лежать на боку',
  'ограничьте употребление алкоголя',
  'поддерживать здоровый вес',
  'массаж',
  'медикаменты',
  'медитация',
  'снизить стресс',
  'расслабляться',
  'удалить коросту влажной сжатой тканью',
  'отдых',
  'соляные ванны',
  'найти помощь',
  'замочите пораженный участок в теплой воде',
  'прекратить употребление алкоголя',
  'остановить кровотечение с помощью давления',
  'ненадолго прекратить есть твердую пищу',
  'остановить раздражение',
  'прекратить принимать наркотики',
  'перейти на свободную одежду',
  'сделать глубокий вдох',
  'принимать безрецептурное болеутоляющее',
  'принимать пробиотики',
  'лечиться радиоактивным йодом',
  'принять вакцину',
  'вдыхать пары',
  'попробовать иглоукалывание',
  'попробуйте делать маленькие глотки воды',
  'использовать антибиотики',
  'использовать чистые тряпки',
  'использовать детол или ним в воде для купания',
  'используйте грелку или холодный компресс',
  'используйте горячую и холодную терапию',
  'используйте лед, чтобы сжать зуд',
  'использовать лимонную мяту',
  'использовать ним в купании',
  'использовать мази',
  'используйте полороидные очки на солнце',
  'использовать компрессию вен',
  'прививка',
  'теплая ванна с английской солью',
  'вымыть руки',
  'мыть руки теплой водой с мылом',
  'наденьте средства индивидуальной защиты, если это возможно']

In [46]:
ru_precs = [val.capitalize() for val in ru_precs]

In [47]:
precs_eng_to_ru = {x[0]:x[1] for x in zip(all_precs,ru_precs)}
precs_ru_to_eng = {x[0]:x[1] for x in zip(ru_precs,all_precs)}

### jsonы

In [48]:
json.dump(precs_eng_to_ru, open("json/precs_eng_to_ru.json", "w"), indent=4, ensure_ascii=False)
json.dump(precs_ru_to_eng, open("json/precs_ru_to_eng.json", "w"), indent=4, ensure_ascii=False)

In [49]:
json.dump(descr_eng_to_ru, open("json/descr_eng_to_ru.json", "w"), indent=4, ensure_ascii=False)
json.dump(descr_ru_to_eng, open("json/descr_ru_to_eng.json", "w"), indent=4, ensure_ascii=False) 

In [50]:
json.dump(disease_eng_to_ru, open("json/disease_eng_to_ru.json", "w"), indent=4, ensure_ascii=False) 
json.dump(disease_ru_to_eng, open("json/disease_ru_to_eng.json", "w"), indent=4, ensure_ascii=False)

In [51]:
json.dump(symp_eng_to_ru, open("json/symp_eng_to_ru.json", "w"), indent=4, ensure_ascii=False)
json.dump(symp_ru_to_eng, open("json/symp_ru_to_eng.json", "w"), indent=4, ensure_ascii=False)

In [52]:
json.dump(di_severity_weights, open("json/severity_weights.json", "w"), indent=4, ensure_ascii=False)

In [53]:
json.dump(di_description, open("json/description.json", "w"), indent=4, ensure_ascii=False)
json.dump(di_precaution, open("json/precaution.json", "w"), indent=4, ensure_ascii=False)

## Предикты

In [54]:
def prediction(model, data: list):
    pred = model.predict([data])[0]
    descr = di_description[pred]
    precaution = di_precaution[pred]
    result = {'Disease':disease_eng_to_ru[pred], 'Description':descr_eng_to_ru[descr], 'Precaution':[precs_eng_to_ru[prec] for prec in precaution]}
    return result

In [55]:
sympotms = [3,5,6,1,3,4,1,1,0,0,0,0,0,0,0,0,0]
prediction(random_forest, sympotms)

{'Disease': 'Псориаз',
 'Description': 'Псориаз — это распространенное кожное заболевание, при котором образуются толстые, красные, неровные пятна, покрытые серебристыми чешуйками. Они могут появиться где угодно, но чаще всего появляются на коже головы, локтях, коленях и нижней части спины. Псориаз не передается от человека к человеку. Иногда это случается у членов одной семьи.',
 'Precaution': ['Мыть руки теплой водой с мылом',
  'Остановить кровотечение с помощью давления',
  'Проконсультируйтесь со специализированным врачом',
  'Соляные ванны']}

In [56]:
prediction(random_forest, [6,4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0])

{'Disease': 'Паралич (кровоизлияние в мозг)',
 'Description': 'Внутримозговое кровоизлияние (ВМК) — это когда кровь внезапно прорывается в мозговую ткань, вызывая повреждение мозга. Симптомы обычно появляются внезапно во время ВЧК. Они включают головную боль, слабость, спутанность сознания и паралич, особенно на одной стороне тела.',
 'Precaution': ['Массаж',
  'Здоровое питание',
  'Упражнения',
  'Проконсультируйтесь со специализированным врачом']}

### Сохранение

In [57]:
filename = 'model1_rf.pkl'
pickle.dump(random_forest, open(f'model/{filename}','wb'))

In [58]:
clff = pickle.load(open(f'model/{filename}','rb'))

In [59]:
prediction(clff, [6,4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0])

{'Disease': 'Паралич (кровоизлияние в мозг)',
 'Description': 'Внутримозговое кровоизлияние (ВМК) — это когда кровь внезапно прорывается в мозговую ткань, вызывая повреждение мозга. Симптомы обычно появляются внезапно во время ВЧК. Они включают головную боль, слабость, спутанность сознания и паралич, особенно на одной стороне тела.',
 'Precaution': ['Массаж',
  'Здоровое питание',
  'Упражнения',
  'Проконсультируйтесь со специализированным врачом']}